<a href="https://colab.research.google.com/github/daisysong76/AI--Machine--learning/blob/main/forecasting_future_purchases_by_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

For forecasting future purchases using sequence prediction models, consider using an LSTM (Long Short-Term Memory) model due to its effectiveness in handling time-series data. After collecting and preprocessing customer transaction history, you train the LSTM model to recognize patterns in purchase sequences. Post-training, evaluate its accuracy using a separate test dataset to assess its predictive performance. Refinement involves tweaking model parameters, like the number of layers or neurons, and retraining until you achieve satisfactory forecast accuracy. This iterative process helps in accurately predicting when and what a customer might buy next, enabling timely and personalized marketing strategies.

start by preprocessing your time-series data (customer purchase history), normalizing it, and then structuring it into sequences for the LSTM to process. Using libraries like TensorFlow or PyTorch, you define an LSTM model, train it on your data, evaluate its performance on a test set, and adjust parameters as needed for improvement

To implement a project forecasting future purchases with sequence prediction models:


Collect Data: Gather historical customer purchase data.


By using web scarpping

In [ ]:
import requests
from bs4 import BeautifulSoup

# Define the URL of the website to scrape
url = 'https://www.example.com/products'

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all product elements on the page
    products = soup.find_all('div', class_='product')

    # Extract relevant information from each product
    for product in products:
        name = product.find('h2', class_='product-name').text.strip()
        price = product.find('span', class_='product-price').text.strip()
        rating = product.find('span', class_='product-rating').text.strip()

        # Save the data to a CSV file or database
        with open('products.csv', 'a') as f:
            f.write(f'{name}, {price}, {rating}\n')

    print('Data collection complete!')
else:
    print('Failed to retrieve data. Status code:', response.status_code)


In this example, we're pretending to scrape product data from an imaginary website and extract information such as product name, price, and rating. The data is then saved to a CSV file for further analysis. Remember to replace 'https://www.example.com/products' with the actual URL of the website you want to scrape. Additionally, always check the website's terms of service and robots.txt file to ensure compliance with web scraping guidelines.

Preprocess Data: Normalize and structure the data into sequences.


In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Load the collected data from the CSV file
data = pd.read_csv('products.csv')

# Normalize the price and rating columns using Min-Max scaling
scaler = MinMaxScaler()
data[['price', 'rating']] = scaler.fit_transform(data[['price', 'rating']])

# Structure the data into sequences
# For example, create sequences of 5 consecutive data points
sequence_length = 5
sequences = []
for i in range(len(data) - sequence_length + 1):
    sequence = data.iloc[i:i+sequence_length]
    sequences.append(sequence.values)

# Convert the sequences to a numpy array
sequences = np.array(sequences)

print('Data preprocessing complete!')


In this example, we're pretending to load the collected product data from a CSV file, normalize the 'price' and 'rating' columns using Min-Max scaling, and structure the data into sequences of consecutive data points. Each sequence contains 5 consecutive data points, but you can adjust the sequence_length variable to fit your specific needs. Finally, the sequences are converted to a numpy array for further processing.

Split Data: Divide data into training and testing sets.


In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into features (X) and target (y)
X = sequences[:, :, :-1]  # All columns except the last one
y = sequences[:, -1, -1]  # Last column (target variable)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print('Data splitting complete!')


In this example, we're pretending to split the preprocessed sequences into features (X) and target (y). The features include all columns except the last one, which represents the target variable (e.g., the next purchase). Then, we split the data into training and testing sets using the train_test_split function from scikit-learn. The testing set size is set to 20% of the total data, and we're using a random state for reproducibility.

Define Model: Create an LSTM model using TensorFlow or PyTorch.


In [ ]:
import torch
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# Example usage:
input_size = 10  # Size of each input sequence
hidden_size = 32  # Number of features in the hidden state of the LSTM
num_layers = 2  # Number of LSTM layers
output_size = 1  # Size of the output (e.g., predicted purchase)

model = LSTMModel(input_size, hidden_size, num_layers, output_size)
print(model)


In this pretend example, we define an LSTM model class called LSTMModel using PyTorch's nn.Module class. The constructor initializes the LSTM and fully connected (linear) layers. The forward method defines the forward pass of the model, where the input sequence is passed through the LSTM layers and the output of the last time step is fed into a fully connected layer to produce the final output. Finally, we create an instance of this model with the desired input, hidden, and output sizes.

Train Model: Train the LSTM on the training set.


In [ ]:
import torch.optim as optim

# Define hyperparameters
learning_rate = 0.001
num_epochs = 10

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:  # Assuming train_loader is your DataLoader
        inputs = inputs.float()  # Convert inputs to float if necessary
        labels = labels.float()  # Convert labels to float if necessary

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Print average loss for the epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader)}')

print('Training complete!')


In this code:

We define hyperparameters such as learning rate and number of epochs.
We define a loss function (mean squared error) and an optimizer (Adam).
We iterate over each epoch and within each epoch, iterate over each batch in the training DataLoader.
We perform a forward pass, compute the loss, perform a backward pass, and update the model parameters.
We print the average loss for each epoch.
Make sure to replace train_loader with your actual DataLoader containing the training data. Additionally, ensure that your input data (inputs) and labels (labels) are appropriately formatted and converted to PyTorch tensors.

Evaluate: Assess the model's performance on the test set.

In [ ]:
# Set the model to evaluation mode
model.eval()

# Define a variable to accumulate the total loss
total_loss = 0.0

# Iterate over the test dataset
with torch.no_grad():
    for inputs, labels in test_loader:  # Assuming test_loader is your DataLoader
        inputs = inputs.float()  # Convert inputs to float if necessary
        labels = labels.float()  # Convert labels to float if necessary

        # Forward pass
        outputs = model(inputs)

        # Compute the loss
        loss = criterion(outputs, labels)

        # Accumulate the total loss
        total_loss += loss.item()

# Calculate the average loss
average_loss = total_loss / len(test_loader)

# Print the average loss
print(f'Average Loss on Test Set: {average_loss}')


In this code:

We set the model to evaluation mode using model.eval().
We define a variable total_loss to accumulate the total loss.
We iterate over each batch in the test DataLoader, performing a forward pass to compute the predicted outputs and calculating the loss between the predicted outputs and the actual labels.
We accumulate the total loss across all batches.
Finally, we calculate the average loss by dividing the total loss by the number of batches in the test DataLoader and print the result.
Make sure to replace test_loader with your actual DataLoader containing the test data. Additionally, ensure that your input data (inputs) and labels (labels) are appropriately formatted and converted to PyTorch tensors.

Refine: Adjust model parameters and retrain for improved accuracy.


In [ ]:
# Define your model architecture with adjusted parameters
refined_model = LSTMModel(input_size, hidden_size, num_layers, output_size)

# Define your loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(refined_model.parameters(), lr=learning_rate)

# Train the refined model
for epoch in range(num_epochs):
    # Set the model to training mode
    refined_model.train()

    # Initialize total loss for this epoch
    total_loss = 0.0

    # Iterate over the training dataset
    for inputs, labels in train_loader:  # Assuming train_loader is your DataLoader
        inputs = inputs.float()  # Convert inputs to float if necessary
        labels = labels.float()  # Convert labels to float if necessary

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = refined_model(inputs)

        # Compute the loss
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()

        # Optimize
        optimizer.step()

        # Accumulate the total loss
        total_loss += loss.item()

    # Calculate the average loss for this epoch
    average_loss = total_loss / len(train_loader)

    # Print the average loss for this epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Average Loss: {average_loss}')

# Evaluate the refined model
# (Code for evaluation is the same as provided in the previous response)

# Optionally, save the refined model
torch.save(refined_model.state_dict(), 'refined_model.pth')


In this code:

We define a new instance of the model (refined_model) with adjusted parameters. You might adjust parameters such as the number of layers, hidden size, or learning rate based on insights gained from previous training.
We define a new loss function and optimizer for the refined model.
We train the refined model similarly to before, iterating over the training dataset for a specified number of epochs, performing forward and backward passes, and updating the model parameters.
After training, we can evaluate the refined model using the same evaluation procedure as before.
Optionally, you can save the refined model's state dictionary for future use. Adjust the file path as needed.
Ensure that you adjust the parameters (input_size, hidden_size, num_layers, output_size, learning_rate, num_epochs) and DataLoader (train_loader) according to your specific requirements and dataset.

Deploy: Use the model to predict future customer purchases.
Load the trained model: Load the saved model weights or instantiate a new instance of the model with the same architecture and load the weights into it.

Prepare input data: Prepare the input data for prediction. This could involve selecting relevant features from your dataset, normalizing the data if necessary, and structuring it into sequences similar to how you did during training.

Perform inference: Use the trained model to perform inference on the input data. Pass the input data through the model and obtain the model's predictions.

Post-processing (optional): Perform any necessary post-processing on the model predictions. This could involve converting the predictions into a human-readable format, mapping them back to original categories or labels, or applying any business logic.

Use predictions: Use the model predictions to make decisions or take actions in your application or business process. For example, you could use the predictions to personalize marketing campaigns, recommend products to users, or optimize inventory management.

In [ ]:
import torch

# 1. Load the trained model
model = LSTMModel(input_size, hidden_size, num_layers, output_size)
model.load_state_dict(torch.load('path_to_saved_model.pth'))
model.eval()

# 2. Prepare input data (replace this with your actual input data preparation code)
input_data = prepare_input_data_for_prediction()

# 3. Perform inference
with torch.no_grad():
    outputs = model(input_data)

# 4. Post-processing (optional)
# Perform any necessary post-processing on the model predictions

# 5. Use predictions
# Use the model predictions for decision-making or further processing


In this code:

Replace 'path_to_saved_model.pth' with the path to your saved model file.
Replace prepare_input_data_for_prediction() with your actual data preparation code to prepare input data for prediction.
The model.eval() call is used to set the model to evaluation mode before performing inference.
The torch.no_grad() context manager is used to disable gradient calculation during inference to reduce memory usage and speed up computation.
Make sure to adjust the code according to your specific use case and requirements.